In [ ]:

from langchain.messages import HumanMessage,SystemMessage
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_classic.output_parsers import PydanticOutputParser
from langchain_community.tools import DuckDuckGoSearchRun
from pydantic import BaseModel
from typing import Optional, Literal

import re

In [454]:
HISTORY = []
def set_history(conv):
    HISTORY.append(conv)

def get_history():
    return HISTORY

In [455]:
search_tool = DuckDuckGoSearchRun()

In [456]:
class AgentResponse(BaseModel):
    type: Literal["action", "final"]
    tool: Optional[str] = None
    input: Optional[str] = None
    answer: Optional[str] = None


In [457]:
parser = PydanticOutputParser(pydantic_object=AgentResponse,try_parse=True)

In [458]:
format_instructions = parser.get_format_instructions()

In [459]:
tools_list = [search_tool]

tools_lib = {}

for tool in tools_list:
    tools_lib[tool.name] = tool
    



In [460]:
SYSTEM = f"""
You are an agent.

You MUST respond in valid JSON only.
Do not include any text outside JSON.

Decide one of the following:

• If a tool is required → type = "action"
• If no tool is required → type = "final"

If you have enough information from previous observations to answer the question, respond with:

  "type": "final",
  "tool": null,
  "input": null,
  "answer": "<your answer here>"
IN DICT FORMART

Do NOT call any tool if you already have enough information.
Available tools:
{list(tools_lib.keys())}

{format_instructions}


"""


In [461]:
prompt_plate = ChatPromptTemplate.from_messages([
    SystemMessage(content=SYSTEM),
    MessagesPlaceholder("HISTORY"),
    HumanMessagePromptTemplate.from_template("{query}")
])


In [462]:
llm = ChatOpenAI(api_key=Key,temperature=0)

In [ ]:
query = ""
set_history(HumanMessage(content=query))

MAX_STEPS = 5
steps = 0

while steps < MAX_STEPS:
    steps += 1

    response = (prompt_plate | llm | parser).invoke({
        "query": query,
        "HISTORY": get_history()
    })

    print("\nAGENT RESPONSE:", response)

  
    if response.type == "action":
        tool_name = response.tool
        tool_input = response.input
        if tool_name not in tools_lib:
            print(f" Unknown tool: {tool_name}")
            break

        print(f"\n Running tool: {tool_name} with input: {tool_input}")
        result = tools_lib[tool_name].invoke(tool_input)
        print("Tool result:", result)

        set_history(HumanMessage(content=f"Observation: {result}"))

    elif response.type == "final":
        print("\n FINAL ANSWER:", response.answer)
        break

else:
    print(" Max steps reached without final answer")



AGENT RESPONSE: type='final' tool=None input=None answer='Parul University hosted the Smart India Hackathon 2025 and celebrated its 9th convocation recently. They also celebrated the 77th Republic Day with Chief Guest Lt. General Shokin Chauhan. Additionally, the university signed an MOU with Vadodara City Police. Industry professionals visited the campus to demonstrate the potential of the Micro-Nano Research and Development Center.'

 FINAL ANSWER: Parul University hosted the Smart India Hackathon 2025 and celebrated its 9th convocation recently. They also celebrated the 77th Republic Day with Chief Guest Lt. General Shokin Chauhan. Additionally, the university signed an MOU with Vadodara City Police. Industry professionals visited the campus to demonstrate the potential of the Micro-Nano Research and Development Center.
